In [7]:
import numpy as np
import openai
import psycopg2

from pgvector.psycopg2 import register_vector
from psycopg2.extras import execute_values
from loguru import logger
from jobspy import scrape_jobs

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#Helper function: get embeddings for a text
def get_embeddings(text):
   response = openai.Embedding.create(
       model="text-embedding-ada-002",
       input = text.replace("\n"," ")
   )
   embedding = response['data'][0]['embedding']
   return embedding

In [ ]:
def etl():
    jobs = scrape_jobs(
        site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
        search_term="data",
        location="San Francisco",
        results_wanted=20,
        hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
        country_indeed='USA',  # only needed for indeed / glassdoor
        # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
    )

    logger.info(f"{jobs.shape}")
    

    # Load
    connection = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="mypass",
        host="localhost",
        port="5432"
    )
    
    cur = connection.cursor()
    #install pgvector
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
    connection.commit()
    
    
    register_vector(connection)
    
    # Create table to store embeddings and metadata
    table_create_command = """
    CREATE TABLE embeddings (
                id bigserial primary key, 
                site text,
                job_url text,
                title text,
                company text,
                location text,
                date_posted text,
                interval text,
                min_amount float,
                max_amount float,
                currency text,
                is_remote text,
                description text,
                company_url text,
                company_logo text,
                embedding vector(1536)
                );
                """
    cur.execute(table_create_command)
    cur.close()
    connection.commit()
    
    #Batch insert embeddings and metadata from dataframe into PostgreSQL database
    register_vector(connection)
    cur = connection.cursor()
    # Prepare the list of tuples to insert
    # Prepare the list of tuples to insert
    
    data_list = [
        (
            row['site'],
            row['job_url'],
            row['title'],
            row['company'],
            row['location'],
            row['date_posted'],
            row['interval'],
            row['min_amount'],
            row['max_amount'],
            row['currency'],
            row['is_remote'],
            row['description'],
            row['company_url'],
            row['company_logo'],
            np.array(get_embeddings(row['description']))  # Assuming 'embeddings' is a column with vector data
        )
        for index, row in jobs.iterrows()
    ]

    # Use execute_values to perform batch insertion
    execute_values(
        cur,
        """
        INSERT INTO embeddings (site, job_url, title, company, location, date_posted, interval, min_amount, max_amount, currency, is_remote, description, company_url, company_logo, embedding)
        VALUES %s
        """,
        data_list
    )
    # Commit after we insert all embeddings
    connection.commit()

In [6]:
etl()

2024-11-11 16:55:20,122 - INFO - JobSpy:Indeed - search page: 1 / 1
2024-11-11 16:55:20,123 - INFO - JobSpy:LinkedIn - search page: 1 / 2
2024-11-11 16:55:20,814 - INFO - JobSpy:ZipRecruiter - search page: 1 / 1
2024-11-11 16:55:21,062 - ERROR - JobSpy:ZipRecruiter - ZipRecruiter response status code 403 with response: {"error_code":"geoblocked-gdpr","error_message":"This app is not available in the European Union, which enforces the General Data Protection Regulation (GDPR). Because of this regulation, ZipRecruiter cannot provide access at this time.","request_id":"CFRAY:8e0e9b9add1c376e-IAD","status_code":403}
2024-11-11 16:55:21,063 - INFO - JobSpy:ZipRecruiter - finished scraping
/home/sina/miniconda3/envs/reflex_jobs/lib/python3.11/site-packages/markdownify/__init__.py:104: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')
2024-11-1